In [1]:
import numpy as np 
import pandas as pd 
from data_processing import generate_vocab, process_data, create_dataloaders 
from model import get_pretrained_emb, EncoderDecoder, EncoderRNN, DecoderRNN, Attention, DecoderAttnRNN
from train_eval import train_and_eval, inspect_model, count_parameters, summarize_results, plot_single_learning_curve
import importlib
import pickle as pkl 

In [2]:
# parameters and hyperparameters 
SRC_LANG = 'zh'
TARG_LANG = 'en'
SRC_VOCAB_SIZE = 30000
TARG_VOCAB_SIZE = 30000
BATCH_SIZE = 32
SRC_MAX_SENTENCE_LEN = 10 
TARG_MAX_SENTENCE_LEN = 10
NUM_EPOCHS = 2000 
LR = 0.0005
NUM_LAYERS = 2 
ENC_HIDDEN_DIM = 300 
DEC_HIDDEN_DIM = 2 * ENC_HIDDEN_DIM

In [3]:
# takes a long time to process, save to pickle for reimport in future 
# vocab = generate_vocab(SRC_LANG, TARG_LANG, SRC_VOCAB_SIZE, TARG_VOCAB_SIZE)
# vocab_filename = "{}-{}-vocab.p".format(SRC_LANG, TARG_LANG)
# pkl.dump(vocab, open(vocab_filename, "wb"))

In [4]:
# reload from pickle 
vocab_filename = "{}-{}-vocab.p".format(SRC_LANG, TARG_LANG)
vocab = pkl.load(open(vocab_filename, "rb"))
data = process_data(SRC_LANG, TARG_LANG, vocab)
limited_data = process_data(SRC_LANG, TARG_LANG, vocab, sample_limit=BATCH_SIZE) 

In [5]:
# create dataloaders 
full_loaders = create_dataloaders(data, SRC_MAX_SENTENCE_LEN, TARG_MAX_SENTENCE_LEN, BATCH_SIZE)
fast_loaders = create_dataloaders(limited_data, SRC_MAX_SENTENCE_LEN, TARG_MAX_SENTENCE_LEN, BATCH_SIZE)

In [6]:
# define model 
encoder = EncoderRNN(enc_hidden_dim=ENC_HIDDEN_DIM, num_layers=NUM_LAYERS, src_max_sentence_len=SRC_MAX_SENTENCE_LEN,
                     pretrained_word2vec=get_pretrained_emb(vocab[SRC_LANG]['word2vec'], vocab[SRC_LANG]['token2id']))
decoder = DecoderRNN(dec_hidden_dim=DEC_HIDDEN_DIM, enc_hidden_dim=ENC_HIDDEN_DIM, num_layers=NUM_LAYERS, 
                     targ_vocab_size=TARG_VOCAB_SIZE, targ_max_sentence_len=TARG_MAX_SENTENCE_LEN, 
                     pretrained_word2vec=get_pretrained_emb(vocab[TARG_LANG]['word2vec'], vocab[TARG_LANG]['token2id']))
# decoder = DecoderAttnRNN(dec_hidden_dim=DEC_HIDDEN_DIM, enc_hidden_dim=ENC_HIDDEN_DIM, num_layers=NUM_LAYERS, 
#                          targ_vocab_size=TARG_VOCAB_SIZE, src_max_sentence_len=SRC_MAX_SENTENCE_LEN, 
#                          targ_max_sentence_len=TARG_MAX_SENTENCE_LEN, 
#                          pretrained_word2vec=get_pretrained_emb(vocab[TARG_LANG]['word2vec'], vocab[TARG_LANG]['token2id']))
model = EncoderDecoder(encoder, decoder, vocab[TARG_LANG]['token2id']) 

In [7]:
train_and_eval(model=model, full_loaders=full_loaders, fast_loaders=fast_loaders, params=params, 
               id2token=vocab[TARG_LANG]['id2token'],
               learning_rate=LR, num_epochs=NUM_EPOCHS, print_intermediate=True, save_checkpoint=True, model_name='test2', 
               lazy_eval=False, lazy_train=True, inspect=True)

Epoch: 0.00, Train Loss: 10.03, Val Loss: 10.22, Train BLEU: 0.53, Val BLEU: 0.21
Inspecting model on training data...
REFERENCE TRANSLATION: 
MODEL TRANSLATION: rumor the the the &apos;s it it it it the the the the the the , , it professions the the the the the the the the the the the the the the the the the professions the the the the the the the the
Inspecting model on validation data...
REFERENCE TRANSLATION: 
MODEL TRANSLATION: the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the &apos;s got got got got got the the the the the the the the the
Epoch: 1.00, Train Loss: 9.67, Val Loss: 10.08, Train BLEU: 0.41, Val BLEU: 0.20
Inspecting model on training data...
REFERENCE TRANSLATION: 
MODEL TRANSLATION: the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the
Inspecting model on validation data...

Epoch: 14.00, Train Loss: 4.41, Val Loss: 8.94, Train BLEU: 0.39, Val BLEU: 0.19
Inspecting model on training data...
REFERENCE TRANSLATION: 
MODEL TRANSLATION: the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the
Inspecting model on validation data...
REFERENCE TRANSLATION: 
MODEL TRANSLATION: the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the
Epoch: 15.00, Train Loss: 4.32, Val Loss: 9.10, Train BLEU: 3.50, Val BLEU: 1.80
Inspecting model on training data...
REFERENCE TRANSLATION: 
MODEL TRANSLATION: <EOS> the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the
Inspecting model on validation data...
REFERENCE TRANS

Epoch: 28.00, Train Loss: 3.85, Val Loss: 10.50, Train BLEU: 4.97, Val BLEU: 1.73
Inspecting model on training data...
REFERENCE TRANSLATION: 
MODEL TRANSLATION: <EOS> the the the it it it it it life the the the the the the the the <EOS> the the the the the the the the the the the the the the the the the <EOS> the the the the the the the the
Inspecting model on validation data...
REFERENCE TRANSLATION: 
MODEL TRANSLATION: <EOS> the the the the the the the the the the the the the the the the the <EOS> the the the the the the the the life the the the the it it it it <EOS> the the the the the the the the
Epoch: 29.00, Train Loss: 3.82, Val Loss: 10.56, Train BLEU: 6.82, Val BLEU: 1.75
Inspecting model on training data...
REFERENCE TRANSLATION: 
MODEL TRANSLATION: <EOS> the the the it it it it it life the the the the the the the the <EOS> the the the the the the the the the the the the the the the the the <EOS> the the the the the the the the
Inspecting model on validation data...
REFERENC

Epoch: 41.00, Train Loss: 3.51, Val Loss: 10.97, Train BLEU: 10.63, Val BLEU: 1.63
Inspecting model on training data...
REFERENCE TRANSLATION: 
MODEL TRANSLATION: <EOS> the the the <EOS> <EOS> &apos;s &apos;s &apos;s life the the the the the the the the <EOS> the the the the the the it it the the the the the the the the it <EOS> the the the the the the it it
Inspecting model on validation data...
REFERENCE TRANSLATION: 
MODEL TRANSLATION: <EOS> the the the the the the it it bill the the the the the the the it is the the the the the it it it life the the the have have it it it <EOS> the the the the the the it it
Epoch: 42.00, Train Loss: 3.49, Val Loss: 10.99, Train BLEU: 10.69, Val BLEU: 1.65
Inspecting model on training data...
REFERENCE TRANSLATION: 
MODEL TRANSLATION: <EOS> the the the <EOS> <EOS> <EOS> &apos;s &apos;s life the the the the the the the the <EOS> the the the the the the it it the the the the the the the the it deep the the the the the the it it
Inspecting model on val

Epoch: 55.00, Train Loss: 3.17, Val Loss: 11.18, Train BLEU: 12.50, Val BLEU: 2.70
Inspecting model on training data...
REFERENCE TRANSLATION: 
MODEL TRANSLATION: <EOS> the the the <EOS> <EOS> <EOS> <EOS> <EOS> life the the the the the the the , deep the the the the the the it it oceans the the the the the , , it deep the the the the the the it it
Inspecting model on validation data...
REFERENCE TRANSLATION: 
MODEL TRANSLATION: deep the the the the the the it it by the the the the the , it it <EOS> the the the the the the to it life tell the the the &apos;s &apos;s &apos;s &apos;s deep the the the the the the it it
Epoch: 56.00, Train Loss: 3.13, Val Loss: 11.23, Train BLEU: 12.06, Val BLEU: 2.36
Inspecting model on training data...
REFERENCE TRANSLATION: 
MODEL TRANSLATION: <EOS> the the and <EOS> <EOS> <EOS> <EOS> &apos;s life the the the the the , &apos;s &apos;s deep the the the the it it it it oceans the the the the , it &apos;s &apos;s deep the the the the to it it it
Inspecting 

...

Epoch: 1953.00, Train Loss: 0.00, Val Loss: 15.88, Train BLEU: 17.03, Val BLEU: 3.22
Inspecting model on training data...
REFERENCE TRANSLATION: 
MODEL TRANSLATION: <EOS> titanic about and and us longest . <EOS> life we is is the the those <EOS> like in you think the the to but but these we we and and , , favorites love <EOS> in you the the the the submarine . all
Inspecting model on validation data...
REFERENCE TRANSLATION: 
MODEL TRANSLATION: <EOS> some the the the but the it it this is average the , voyage , animals these is that oceans oceans are to <EOS> it these life tell most most we we one ? &apos;s in you think the the the . but .
Epoch: 1954.00, Train Loss: 0.00, Val Loss: 15.88, Train BLEU: 17.03, Val BLEU: 3.20
Inspecting model on training data...
REFERENCE TRANSLATION: 
MODEL TRANSLATION: <EOS> titanic about and and us longest . <EOS> life we is is the the those <EOS> like in you think the the to but but these we we and and , , favorites love <EOS> in you the the the the s

Epoch: 1966.00, Train Loss: 0.00, Val Loss: 15.88, Train BLEU: 17.03, Val BLEU: 3.20
Inspecting model on training data...
REFERENCE TRANSLATION: 
MODEL TRANSLATION: <EOS> titanic about and and us longest <EOS> got life we is is the the those <EOS> like in you think the the to but but these we we and and , , favorites love <EOS> in you the the the the submarine . all
Inspecting model on validation data...
REFERENCE TRANSLATION: 
MODEL TRANSLATION: <EOS> some the the the but the it it this is average the , voyage , animals these is that oceans oceans are to <EOS> it these life tell most most we we one ? &apos;s in you think the the the . but .
Epoch: 1967.00, Train Loss: 0.00, Val Loss: 15.89, Train BLEU: 17.03, Val BLEU: 3.21
Inspecting model on training data...
REFERENCE TRANSLATION: 
MODEL TRANSLATION: <EOS> titanic about and and us longest <EOS> got life we is is the the those <EOS> like in you think the the to but but these we we and and , , favorites love <EOS> in you the the the t

Epoch: 1979.00, Train Loss: 0.00, Val Loss: 15.89, Train BLEU: 17.03, Val BLEU: 3.23
Inspecting model on training data...
REFERENCE TRANSLATION: 
MODEL TRANSLATION: <EOS> titanic about and and us longest . <EOS> life we is is the the those <EOS> like in you think the the to but but these we we and and , , favorites love <EOS> in you the the the the submarine . all
Inspecting model on validation data...
REFERENCE TRANSLATION: 
MODEL TRANSLATION: <EOS> some the the the but the it it this is average the , voyage , animals these is that oceans oceans are to <EOS> it these life tell most most we we one ? &apos;s in you think the the the . but .
Epoch: 1980.00, Train Loss: 0.00, Val Loss: 15.89, Train BLEU: 17.03, Val BLEU: 3.23
Inspecting model on training data...
REFERENCE TRANSLATION: 
MODEL TRANSLATION: <EOS> titanic about and and us longest . <EOS> life we is is the the those <EOS> like in you think the the to but but these we we and and , , favorites love <EOS> in you the the the the s

Epoch: 1992.00, Train Loss: 0.00, Val Loss: 15.90, Train BLEU: 17.03, Val BLEU: 3.20
Inspecting model on training data...
REFERENCE TRANSLATION: 
MODEL TRANSLATION: <EOS> titanic about and and us longest . <EOS> life we is is the the those <EOS> like in you think the the to but but these we we and and , , favorites love <EOS> in you the the the the submarine . all
Inspecting model on validation data...
REFERENCE TRANSLATION: 
MODEL TRANSLATION: <EOS> some the the the but the it it this is average the , voyage , animals these is that oceans oceans are to <EOS> it these life tell most most we we one ? &apos;s in you think the the the . but .
Epoch: 1993.00, Train Loss: 0.00, Val Loss: 15.90, Train BLEU: 17.03, Val BLEU: 3.20
Inspecting model on training data...
REFERENCE TRANSLATION: 
MODEL TRANSLATION: <EOS> titanic about and and us longest . <EOS> life we is is the the those <EOS> like in you think the the to but but these we we and and , , favorites love <EOS> in you the the the the s

[{'epoch': 0.0,
  'train_bleu': 0.5327557365316998,
  'train_loss': 10.03071403503418,
  'val_bleu': 0.21492448036333736,
  'val_loss': 10.217119216918945},
 {'epoch': 1.0,
  'train_bleu': 0.4060659547484828,
  'train_loss': 9.673993110656738,
  'val_bleu': 0.19658953498563464,
  'val_loss': 10.081694602966309},
 {'epoch': 2.0,
  'train_bleu': 0.39428335960173344,
  'train_loss': 9.168869972229004,
  'val_bleu': 0.18915755769199444,
  'val_loss': 9.870261192321777},
 {'epoch': 3.0,
  'train_bleu': 0.39428335960173344,
  'train_loss': 8.553747177124023,
  'val_bleu': 0.18915755769199444,
  'val_loss': 9.60238265991211},
 {'epoch': 4.0,
  'train_bleu': 0.39428335960173344,
  'train_loss': 7.9334187507629395,
  'val_bleu': 0.18915755769199444,
  'val_loss': 9.33469295501709},
 {'epoch': 5.0,
  'train_bleu': 0.39428335960173344,
  'train_loss': 7.349544525146484,
  'val_bleu': 0.18915755769199444,
  'val_loss': 9.095149993896484},
 {'epoch': 6.0,
  'train_bleu': 0.39428335960173344,
  'tra